In [8]:
import os
import logging
from pathlib import Path

import boto3
from boto3 import client
from botocore.exceptions import ClientError

In [9]:
class S3:
    def __init__(self):
        self.s3_client = boto3.client('s3')
        
    def create_bucket(self, bucket_name, region = None):
        try:
            if region is None:
                s3_client = self.s3_client
                s3_client.create_bucket(Bucket=bucket_name)
            else:
                s3_client = boto3.client('s3', region_name = region)
                location = {'LocationConstraint': region}
                s3_client.create_bucket(Bucket = bucket_name,
                                        CreateBucketConfiguration = location)
        except ClientError as e:
            logging.error(e)
            return False
        return True

    def get_buckets(self):
        buckets = []
        response = self.s3_client.list_buckets()
        for bucket in response['Buckets']:
            buckets.append(bucket["Name"])
        return buckets

    def get_file_folders(self, bucket_name, prefix = ''):
        file_names = []
        folders = []

        default_kwargs = {
            'Bucket': bucket_name,
            'Prefix': prefix
        }
        next_token = ''

        while next_token is not None:
            updated_kwargs = default_kwargs.copy()
            if next_token != '':
                updated_kwargs['ContinuationToken'] = next_token

            response = self.s3_client.list_objects_v2(**default_kwargs)
            contents = response.get('Contents')
            if contents is None:
                return None, None
            for result in contents:
                key = result.get('Key')
                if key[-1] == '/':
                    folders.append(key)
                else:
                    file_names.append(key)

            next_token = response.get('NextContinuationToken')

        return file_names, folders

    def download_files(self, bucket_name, local_path, file_names, folders):
        local_path = Path(local_path)

        for folder in folders:
            folder_path = Path.joinpath(local_path, folder)
            folder_path.mkdir(parents = True, exist_ok = True)

        for file_name in file_names:
            file_path = Path.joinpath(local_path, file_name)
            file_path.parent.mkdir(parents = True, exist_ok = True)
            self.s3_client.download_file(
                bucket_name,
                file_name,
                str(file_path)
            )

    def upload_file(self, file_name, bucket, object_name = None):
        if object_name is None:
            object_name = os.path.basename(file_name)   
        try:
            response = self.s3_client.upload_file(file_name, bucket, object_name)
        except ClientError as e:
            logging.error(e)
            return False
        return True

    def print_buckets(self):
        response = self.s3_client.list_buckets()

        print('Existing buckets:')
        for bucket in response['Buckets']:
            print(f'  {bucket["Name"]}')
            
    def print_files_in_buckets(self, bucket_name):
        s3 = boto3.resource('s3')
        my_bucket = s3.Bucket(bucket_name)

        for object_summary in my_bucket.objects.filter():
            print(f'-- {object_summary.key}')
        print()

In [10]:
s3 = S3()
s3.print_buckets()
print()
# for bucket in get_buckets(s3):
#     print('Bucket Name:', bucket)
#     print_files_in_buckets(bucket)

Existing buckets:
  dev-bluebox
  tf-poc-aws-state
  ts-whisper-data
  tsadobe
  tsdigiseg
  tsivt
  tsliveramp
  tsppiq
  tstruoptik



In [12]:
bucket = 'ts-whisper-data'
audio_data_path = '../data/audio/'
s3 = S3()
audio_files = os.listdir(audio_data_path)
for audio_file in audio_files:
    s3.upload_file(os.path.join(audio_data_path, audio_file), bucket)
print('Uploaded {} files to bucket: {}'.format(len(audio_files), bucket))

Uploaded 3 files to bucket: ts-whisper-data
